In [37]:
from pathlib import Path 
import numpy as np
import h5py

measurements_path = Path("../data/measurements.hdf5")
processed_path = Path("../data/processed.hdf5")

dset_opt = dict(dtype=np.float32, compression='gzip')

with (h5py.File(processed_path, "w") as db_processed,
      h5py.File(measurements_path, "r") as db_measurements):
  for driver in db_measurements:
    print(driver)
    db_processed.create_group(driver)

    zero_grp = db_measurements[driver]['zeros']
    sig0 = zero_grp['rec'][:, :, 0, 0]
    means = sig0.mean(axis=1)
    stds = sig0.std(axis=1)

    gen = (signame for signame in db_measurements[driver] if signame != 'zeros')
    for signame in gen:
      meas_sig_grp = db_measurements[driver][signame]
      proc_sig_grp = db_processed[driver].create_group(signame)
      
      proc_sig_grp.create_dataset(
        "rec", data=np.median(meas_sig_grp['rec'], axis=-1) - means[:, None, None], 
        **dset_opt)
      proc_sig_grp.attrs['levels'] = meas_sig_grp.attrs['levels']
      proc_sig_grp.attrs['sr'] = meas_sig_grp.attrs['sr']

      print(signame)